# Prompt engineering on worst econ questions: answer verification, temperature, red/blue, context

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from ollama_models import ollama_models
models = ollama_models()

In [3]:
models

['mistral-small3.2:24b-instruct-2506-q4_K_M']

In [4]:
model = models[0]

In [5]:
from load_saved_questions import load_saved_questions
from call_local_llm import call_local_llm

In [6]:
question = load_saved_questions([17763])[0]

In [7]:
    from community_forecast import community_forecast
    import pandas as pd
    from flatten_dict import flatten_dict
    from datetime import datetime
    from gather_research_and_set_prompt import gather_research_and_set_prompt
    from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
    from tqdm import tqdm
    import time

In [8]:
    print('START model', model, 'id', id)
    start_time = time.time()
    questions = [question]
    id = question.id_of_question
    id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}
    df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
    df['id_of_post'] = [q.id_of_post for q in questions]
    df['id_of_question'] = [q.id_of_question for q in questions]
    df['question_options'] = df['question_options'].apply(repr)
    df['today'] = datetime.now().strftime("%Y-%m-%d")
    df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)

START model mistral-small3.2:24b-instruct-2506-q4_K_M id <built-in function id>


In [9]:
    df1 = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title',
        'question_resolution_criteria', 'question_fine_print', 'question_type', 
         'question_description',
        'question_options', 'question_group_variable', 'question_question_weight',
        'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
        'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']].copy()

In [10]:
    df2, rag = gather_research_and_set_prompt(df1, True, model)

Loaded existing index from forecast_index.faiss
Index contains 5839 vectors at initialization


In [22]:
df3 = df2[['id_of_question', 'title',
        'question_resolution_criteria', 'question_type', 
         'question_description','crowd',
          'research', 'asknews', 'learning']].copy()

In [23]:
row = df3.iloc[0]

In [24]:
row

id_of_question                                                              17763
title                           Will Threads report more monthly active users ...
question_resolution_criteria    This question resolves as **YES** if, before D...
question_type                                                              binary
question_description            Mark Zuckerberg launched a twitter competitor,...
crowd                                                                        0.01
research                        # Forecasting Analysis: Threads vs. Twitter Mo...
asknews                         Here are the relevant news articles:\n\n**Four...
learning                                                                       []
Name: 0, dtype: object

In [25]:
for item, value in row.items():
    print(item.upper())
    print()
    print(value)
    print('===============================')
    print()

ID_OF_QUESTION

17763

TITLE

Will Threads report more monthly active users than Twitter before 2026?

QUESTION_RESOLUTION_CRITERIA

This question resolves as **YES** if, before Dec 31, 2025, Meta publicly reports Threads has a Monthly Active User number higher than the best public estimates of Twitter's MAUs at that time, and **NO** otherwise.

QUESTION_TYPE

binary

QUESTION_DESCRIPTION

Mark Zuckerberg launched a twitter competitor, Threads, on July 5, 2023. On the first day after launch, Zuckerberg [claimed](https://www.abc.net.au/news/2023-07-06/millions-sign-up-to-meta-twitter-rival-app-threads-in-hours/102572568) 30 million users signed up, and after the first week, [100 million had](https://www.cnn.com/2023/07/19/tech/threads-meta-growth-plan/index.html), though it is hard to estimate how many of those users will be "monthly active" until some months pass. Many of these users were able to sign up with just one or two clicks from Instagram. 

As of December 2022, Twitter had [36

In [ ]:
row.question_resolution_criteria

In [29]:
prompt2 = f"""
You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
{row.title}
```

```news
{row.asknews}

```research
{row.research}
```

The question defines an event and resolves as YES if the event occurs, otherwise NO. 
You are forecasting the probability of YES.
The event is described by the RESOLUTION CRITERIA.

```resolution_criteria
{row.question_resolution_criteria}
```

The last thing you write is your final answer as: "Probability: ZZ%", 0-100
"""

In [30]:
print(prompt2)


You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
Will Threads report more monthly active users than Twitter before 2026?
```

```news
Here are the relevant news articles:

**Four AI Native Apps with Over 1 Billion Monthly Active Users: Huawei's Xia Yi Ranks Second**
According to QuestMobile data, as of March 2025, the number of monthly active users for mobile native apps reached 591 million. Among them, AI native apps led by six major smartphone manufacturers have emerged as a new force, with a monthly active user scale of 481 million. Specifically, the top 15 native apps with the most active users include six smartphone manufacturers. Huawei's Xia Yi and OPPO's Xiao Bu Assistant ranked second and third, respectively, with 157 million and 148 million monthly active users, surpassing Dou Bao's 115 million and trailing only DeepSeek's 193 million. In contrast to 

In [31]:
answer_mistral = call_local_llm(prompt2, model)

answer_mistral2 = call_local_llm(prompt2, model)

answer_mistral3 = call_local_llm(prompt2, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 3.1944202979405723
START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 2.8885836601257324
START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 3.13161031405131


In [32]:
from extract_forecast import extract_probability_from_response_as_percentage_not_decimal

In [33]:
forecasts = [extract_probability_from_response_as_percentage_not_decimal(x)/100.0
             for x in [answer_mistral, answer_mistral2, answer_mistral3]]

In [34]:
forecasts

[0.3, 0.65, 0.3]

In [35]:
import numpy as np

In [37]:
forecast = float(np.median(forecasts))

In [38]:
forecast

0.3

In [39]:
prompt = f"""
Summarize the gist pf the rationale or thinking of the following answers from 3 different forecasters to a single problem. 

```forecast 1
{answer_mistral}
```

```forecast 2
{answer_mistral2}
```

```forecast 3
{answer_mistral3}
```

DO NOT REFER TO THE 3 FORECASTERS.  PRESENT THIS AS YOUR OWN THINKING, YOUR OWN RATIONALE.  Use as your final the median forecast which is

```median forecast
{forecast}
```
"""

In [40]:
rationale = call_local_llm(prompt, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 1.3061020175615947


In [41]:
print(rationale)

The likelihood that Threads will surpass Twitter in monthly active users by the end of 2025 is assessed at **30%**. This moderate probability takes into account several key factors:

1. **Threads' Initial Growth and Meta's Resources**: Threads has shown strong early momentum, supported by Meta's substantial resources and ecosystem advantages. This could help it attract and retain users over time.

2. **Twitter's Challenges**: Twitter is facing potential user decline under its current leadership, which might make it easier for Threads to gain ground.

3. **Significant Hurdles**: Despite these advantages, Threads still faces major challenges, particularly in user retention and achieving feature parity with Twitter. These issues could slow its growth and make it difficult to surpass Twitter within the specified timeframe.

4. **Time Required**: Building a user base large enough to overtake Twitter's established network will take time, and there is no guarantee that Threads will overcome t

In [42]:
row.forecast = rationale

In [43]:
row.prediction = forecast

In [44]:
row.crowd

0.01

In [45]:
from error import error

In [46]:
error(row)

0.29